# `InfiniteOpt.jl`: The Basics
Now that we are familiarized with Julia and `JuMP.jl`, let's talk about how `InfiniteOpt.jl` builds upon `JuMP.jl` to allow us to intuitively model infinite-dimensional optimization problems.

## Resources
Again, there is only so much that we can cover in a tutorial. Helpful resources include:
- The documentation: https://pulsipher.github.io/InfiniteOpt.jl/stable/.
- The paper: https://www.sciencedirect.com/science/article/pii/S0098135421003458?via%3Dihub 
- The open-source paper: https://arxiv.org/abs/2106.12689
- The `InfiniteOpt.jl` forum: https://github.com/pulsipher/InfiniteOpt.jl/discussions

## Optimal Control in `JuMP.jl`
To motivate `InfiniteOpt.jl`, let's consider modeling a simple optimal control problem in `JuMP.jl`.

### The Formulation
We'll take a look at a trajectory control problem:
$$
\begin{aligned}
&&\underset{x(t), v(t), u(t)}{\text{min}} &&& \int_{t \in \mathcal{D}_t} ||u(t)||_2^2 dt  \\
&&\text{s.t.} &&& \frac{dx}{dt} = v(t), && t \in \mathcal{D}_t\\
&&&&& \frac{dv}{dt} = u(t), && t \in \mathcal{D}_t\\
&&&&& x(t) = xw(t), && t \in \mathcal{D}_{tw} \\
&&&&& v(0) = 0
\end{aligned}
$$
Here position $x(t) \in \mathbb{R}^2$ and velocity $v(t) \in \mathbb{R}^2$ are state variables. The control variable $u(t) \in \mathbb{R}^2$ controls the acceleration (thrust) of the vehicle. Here we wish to plan a path that passes through all the waypoints $xw_i$ and minimizes the thrust used.

We cannot directly model this with `JuMP.jl` because the variables are functions of time $t$ and the formulation contains an integrals/derivatives. This is an infinite-dimensional problem.

### Discretize
We can transform it into a finite optimization `JuMP.jl` can handle by discretizing the variables over time and applying appropriate approximations to the integral/derivatives. 

We will transform the model via:
- let $\mathcal{D}_t = [0, T]$ and choose $n$ time steps such that $\Delta t = \frac{T}{n}$
- discretize each infinite variable (e.g., $\{x_t: t \in \{0, 1, \dots, n\}$)
- replace the integral with a sum over time
- approximate the derivatives via implicit Euler

With these steps, we obtain:
$$
\begin{aligned}
&&\underset{x_t, v_t, u_t}{\text{min}} &&& \sum_{t \in \{1, \dots, n\}} u_{1,t}^2 + u_{2,t}^2  \\
&&\text{s.t.} &&& x_{i,t+1} = x_{i,t} + \Delta t v_{i,t+1}, && i \in I, t \in \{0, 1, \dots, n-1\}\\
&&&&& v_{i,t+1} = v_{i,t} + \Delta t u_{i,t+1}, && i \in I, t \in \{0, 1, \dots, n-1\}\\
&&&&& x_{i,t} = xw_{i,t}, && i \in I, t \in \mathcal{D}_{tw} \\
&&&&& v_{i,0} = 0, && i \in I \\
\end{aligned}
$$

This is a discrete time model that we can formulate with `JuMP.jl`, let's try it out!

### Exercise: Optimal Control in `JuMP.jl`
**Problem**
- Implement the above model in `JuMP.jl`
- Complete the code below

In [ ]:
using JuMP, HiGHS

# Set the parameters
n = 60
Δt = 60 / n 
I = 1:2

# Set the waypoint data
p = [1 4 6 1; 1 3 0 1]
Dtw = [0, 25, 50, 60]
xw = JuMP.Containers.DenseAxisArray(p, I, Dtw)

# Define the model (finish using the HiGHS optimizer)
model = 

# Add the variables (finish for each)
@variable() # x[i, t], i ∈ I, t ∈ {0, 1, ..., n}
@variable() # v[i, t], i ∈ I, t ∈ {0, 1, ..., n}
@variable() # u[i, t], i ∈ I, t ∈ {1, ..., n}

# Add the objective (finish)
@objective() # minimize Σ_{t ∈ 1:n} u[1, t]^2 + u[2, t]^2

# Add the constraints (finish)
@constraint() # x[i, t+1] = x[i, t] + Δt v[i, t+1], i ∈ I, t ∈ {0, 1, ..., n-1}
@constraint() # v[i, t+1] = v[i, t] + Δt u[i, t+1], i ∈ I, t ∈ {0, 1, ..., n-1}
@constraint() # x[i, t] = xw[i, t], i ∈ I, t ∈ Dtw

# Add the initial conditions (finish)
fix.() # set v(0) = 0 hint: recall we can broadcast over a vector of variables

# Optimize the model
optimize!(model)

# Get the results (finish)
if has_values(model)
    x_opt = # extract the values of x 
end;


In [ ]:
using Plots

# Plot the path
scatter(xw[1,:].data, xw[2,:].data, label = "Waypoints")
plot!(x_opt[1, :].data, x_opt[2, :].data, label = "Trajectory")
xlabel!("x_1")
ylabel!("x_2")

This works for this simple case, but we can start to think about some limitations:
- What if we want to implement a non-uniform grid?
- What about implementing (and switching between) derivative approximations (e.g., orthogonal collocation)?
- What about higher fidelity integral approximations?
- What about managing differing grid point between varied integral/derivative approximation schemes?
- What if we want to solve our model without discretization?
- What if we want to add uncertainty and/or PDE constraints?

We could resolve the majority of the above points by manually deriving a finite formulation for `JuMP.jl` each time, but this is cumbersome and prone to error. Hence, it is common to define the model once in discrete time and call it good.

However, characterizing models in discretized forms makes it likely for us to miss the insights we can gain from the infinite-dimensional form of the problem.

### Modeling in `InfiniteOpt.jl`
`InfiniteOpt.jl` is built upon our unifying abstraction for infinite-dimensional optimization. Moreover, it leverages `JuMP`'s architecture to have an intuitive interface.

Let's model the hovercraft optimal control problem in its native infinite-dimensional form using `InfiniteOpt.jl`:

In [ ]:
using InfiniteOpt, HiGHS

# Set the parameters
v0 = zeros(2)
I = 1:2
p = [1 4 6 1; 1 3 0 1]
Dtw = [0, 25, 50, 60]
xw = JuMP.Containers.DenseAxisArray(p, I, Dtw)

# Create the model
model = InfiniteModel(HiGHS.Optimizer)

# Decalare t as an infinite parameter
@infinite_parameter(model, t ∈ [0, 60], num_supports = 61)

# Add the infinite variables
@variable(model, x[I], Infinite(t))
@variable(model, v[I], Infinite(t))
@variable(model, u[I], Infinite(t))

# Set the objective
@objective(model, Min, ∫(u[1]^2 + u[2]^2, t))

# Add the constraints
@constraint(model, ∂.(x, t) .== v)
@constraint(model, ∂.(v, t) .== u)
@constraint(model, [i ∈ I], v[i](0) == v0[i])
@constraint(model, [i ∈ I, tw ∈ Dtw], x[i](tw) == xw[i, tw])

# Optimize the model
optimize!(model)

# Get the results
if has_values(model)
    x_opt = value.(x)
end


In [ ]:
using Plots

scatter(xw[1,:].data, xw[2,:].data, label = "Waypoints")
plot!(x_opt[1], x_opt[2], label = "Trajectory")
xlabel!("x_1")
ylabel!("x_2")

We were able to express the model in its infinite form and `InfiniteOpt.jl` took care of the rest!

### What did we just do?
We will walk through each step of what we just did and learn about how `InfiniteOpt.jl` works.

First, we created a model object using `InfiniteModel`:

In [ ]:
model = InfiniteModel(HiGHS.Optimizer)

We provide the `InfiniteModel` with an optimizer (one supported by `JuMP`) that will ultimately be used by the optimizer model backend to solve the model. More on that later...

Next, following our abstraction, we declare time $t \in [0, 60]$ as an infinite parameter using `@infinite_parameter` which follows a syntax similar to `@variable`:

In [ ]:
@infinite_parameter(model, t ∈ [0, 60], num_supports = 61)

The `num_supports` keyword tells the backend optimizer model we would like to use `61` support (discretization) points to solve the model. We discuss more about the backends and the arguments they take later on.

Now that `t` is defined, let's add the infinite variables using `@variable`:

In [ ]:
@variable(model, x[I], Infinite(t))
@variable(model, v[I], Infinite(t))
@variable(model, u[I], Infinite(t))

Here, the syntax for `@variable` is exactly the same as what we learned for `JuMP`. The only extra thing we do is add the `Infinite` tag containing the infinite parameters the variable depends on.

We define objective using `@objective` in like manner to `JuMP`:

In [ ]:
@objective(model, Min, ∫(u[1]^2 + u[2]^2, t)) # ∫ is unicode from \int

Notice we are able to directly express the integral using `∫` (we can also use `integral`) with respect to the infinite parameter `t`. Its domain is inferred, but we can specify a subdomain and the approximation method we would like to use; more on that later.

Now we define ODEs using `@constraint`, expressing the derivatives with `∂` (can also use `deriv`) using `JuMP`'s broadcasting syntax for convenience:

In [ ]:
@constraint(model, ∂.(x, t) .== v)
@constraint(model, ∂.(v, t) .== u)

All the same constraint types and forms supported by `JuMP.jl`, are also supported in `InfiniteOpt.jl`!

We also define the initial condition:

In [ ]:
@constraint(model, [i ∈ I], v[i](0) == v0[i])

Notice that we can enforce this point constraint by calling `v[i](0)`, this creates a point variable of `v[i](t)` at time 0.

Next, we'll add the waypoint constraint:

In [ ]:
@constraint(model, [i ∈ I, tw ∈ Tw], x[i](tw) == xw[i, tw])

Again we use the point variable syntax to enforce `x` at the waypoint time points `tw`.

With our model defined, we can interrogate it via pretty printing it using `latex_formulation`:

In [ ]:
latex_formulation(model)

In contrast to working with finite-time models, we can feasibly print interpretable models for real-world problems since the infinite-dimensional form is typically much more compact.

With the model all setup, we can optimize it via `optimize!`:

In [ ]:
optimize!(model)

Behind the scenes, our infinite-dimensional model was transformed into a `JuMP.jl` model (the optimizer model backend) via the default transformation method (direct transcription using the 61 supports specified above). We can take a quick sneak peak at the optimizer model:

In [ ]:
optimizer_model(model)

It is indeed a `JuMP.jl` model that uses the `HiGHS` optimizer (it also has some extra stuff to help it integrate seamlessly with the `InfiniteModel`). Note that pretty printing this discrete model would be impractical.

Finally, to complete this example we query the results using `value`:

In [ ]:
value.(x)

Notice, we get a vector of values for each infinite variable `x[i](t)`. These correspond to the discrete time values used in the reformulation which we can also retrieve using `value`:

In [ ]:
value(t)

We can also make other queries as we typically do with `JuMP` models. For instance,

In [ ]:
@show termination_status(model)
@show primal_status(model)
@show solve_time(model)
@show objective_value(model);

## The Building Blocks of `InfiniteModel`s
In this section, we go over the syntax for the modeling objects used in `InfiniteModel`s as per our unifying abstraction. 

Note that many of these constructs such as variables, expressions, objectives, constraints, etc. inherit all capabilities that `JuMP.jl`. `InfiniteOpt.jl` extends these to have additional capabilities needed for InfiniteOpt problems. Hence, our discussion will focus on these additional features.

### Infinite Models
The object behind it all, is the `InfiniteModel` itself. Typically, we will specify this with the optimizer we wish the default optimizer model backend (`TranscriptionOpt`) to use. We'll discuss optimizer models further below, but note that the following definitions are equivalent:

In [ ]:
model = InfiniteModel(HiGHS.Optimizer, add_bridges = false)

model = InfiniteModel()
set_optimizer_model(model, TranscriptionModel(HiGHS.Optimizer, add_bridges = false))

A `TranscriptionModel` is an augmented `JuMP.Model`. Note that the arguments given to `InfiniteModel` are simply forwarded to the optimizer model. We can also provide the constructor of the optimizer backend directly: 

In [ ]:
model = InfiniteModel(HiGHS.Optimizer, OptimizerModel = TranscriptionModel)

This is not necessary in this case since `TranscriptionModel` is the default `OptimizerModel`.

We can set the optimizer and attributes in like manner to `JuMP`:

In [ ]:
model = InfiniteModel()
set_optimizer(model, HiGHS.Optimizer)
set_silent(model)
set_optimizer_attribute(model, "presolve", "on")

### Finite Parameters
As we saw, `JuMP.jl` has limited support for using changeable parameters. `InfiniteOpt.jl` provides finite parameters whose values can be changed and which can be used in any expression. 

These are created via `@finite_parameter` which follows a similar syntax to `JuMP.jl`'s `@NLparameter`:

In [ ]:
@finite_parameter(model, p == 42)
@finite_parameter(model, ps[i ∈ 1:2] == i)

However, unlike `@NLexpression`, these parameters can be used anywhere in the model. We can query and modify the values with `value` and `set_value`, respectively:

In [ ]:
@show value(p)

set_value(p, 10)

@show value(p);

We can also delete parameters from the model:

In [ ]:
delete(model, ps[1])

For more information (including anonymous definition) see https://pulsipher.github.io/InfiniteOpt.jl/stable/guide/finite_parameter/

### Infinite Parameters
Infinite parameters serve as the core entity for defining infinite-dimensional modeling objects. They parameterize the domain of the problem (called the infinite domain). These are defined via `@infinite_parameter`:

In [ ]:
@infinite_parameter(model, t ∈ [0, 10])

This will follow the syntax of `infinite_parameter(s)` in `infinite_domain`. In optimal control, we will typically be concerned time `t`, but we can also define other domain types like spatial position $x \in [-1, 1]^2$:

In [ ]:
@infinite_parameter(model, x[1:2] in [-1, 1], independent = true, num_supports = 42, derivative_method = OrthogonalCollocation(3))

Notice we can pass some keyword arguments as well:
- `independent` indicates whether the parameters are independent such that their domain is computed via the Cartesian product
- `num_supports` is an argument for the optimizer model indicating how many supports we'd like to use
- `derivative_method` is another optimizer model argument indicating how we want to approximate the derivatives that depend on these parameters.

We can also define random parameters with any distribution from `Distributions.jl`. For instance,

In [ ]:
using Distributions, LinearAlgebra

μ = zeros(4); Σ = I(4)
@infinite_parameter(model, ξ[1:4] ~ MvNormal(μ, Σ), num_supports = 1000)

This enables us to tackle stochastic programs as well!

We can also do some optimizer model related changes after definition. For instance:

In [ ]:
add_supports(t, [0, 0.3, 4, 10])
set_derivative_method(t, FiniteDifference(Backward()))

Finally, we can delete infinite parameters if we want to:

In [ ]:
delete(model, ξ)

#### Exercise: Create an Infinite Parameter
**Problem**
- Create an infinite parameter $\ell \in [-1, 1]$
- Specify that it should use 10 supports

In [ ]:
# PUT CODE HERE


For more information (including anonymous definition) see https://pulsipher.github.io/InfiniteOpt.jl/stable/guide/parameter/

### Parameter Functions
Sometimes we may wish to embed some arbitrary Julia function of infinite parameters into our model. This is often the case with setpoints in optimal control and experimental data in parameter estimation.

While it is sometimes possible to represent these as explicit algebraic expressions, it is not always convenient or possible. To this end we provide, parameter functions via `@parameter_function` which follows a 
`JuMP`-like syntax:

In [ ]:
function setpoint(t)
    if t <= 2
        return 1.2
    elseif t <= 5
        return 3.0
    elseif t <= 7
        return 1.6
    else
        return 3.5
    end
end

@parameter_function(model, mysetpoint == setpoint(t))

We can then use `mysetpoint` in any expression in `InfiniteOpt.jl`, and it will be handled appropriately when the model is transformed. Note that functions must return a scalar number.

Sometimes it is also convenient to use the functional definition API `parameter_function` with a Julia `do` block:

In [ ]:
mysetpoint = parameter_function(t, name = "setpoint") do t
    if t <= 5
        return 2.0
    else 
        return 10.2
    end
 end

We can also define a collection of parameter functions that depend on some parent function:

In [ ]:
parent(t, a) = sin(t)^a

@parameter_function(model, pfunc[i = 1:3] == t -> parent(t, i))

For more information (including anonymous definition) see https://pulsipher.github.io/InfiniteOpt.jl/stable/guide/expression/#par_func_docs.

### Variables
All variables can be made with `@variable` just like any `JuMP.jl` variable, the only difference is that we'll add an appropriate "tag" to it.

Let's begin with the most common: infinite variables.

These use the `Infinite` tag as we have already seen:

In [ ]:
@variable(model, y >= 0, Infinite(t))

Notice we can add bounds and conditions like normal.

Uniquely, with infinite variables we can provide a start (guess) that a function with the same input dimensions as the variable. For instance:

In [ ]:
@variable(model, -1 <= w[i = 1:2] <= 1, Infinite(t), start = sin)
model[:w] # macro objects are registered like they are in JuMP

Each $w_i(t)$ is set with an initial guess trajectory of $\sin(t)$.

Next, semi-infinite variables correspond to infinite variables whose domain is partially evaluated. We can define these using the `SemiInfinite` tag or via our functional syntax:

In [ ]:
@variable(model, q, Infinite(t, x)) # an infinite variable 

@variable(model, q0, SemiInfinite(q, 0, x)) # macro definition

q0 = q(0, x) # functional definition

Here `q0` acts as an alias Julia variable. Note that simply invoking the functional syntax at different time point will have intuitive printing:

In [ ]:
q(1, x)

Similarly, we can define point variables using the convenient functional syntax:

In [ ]:
q(0, [-1, 1])
y(10)

Alternatively, we can use `@variable` with the `Point` tag:

In [ ]:
@variable(model, y0, Point(y, 0))

Finally, we can define finite variables by simply using the `@variable` as one normally would in `JuMP.jl`: 

In [ ]:
@variable(model, 0 ≤ z ≤ 4, Int)

Moreover, variables can be modified/queried/deleted in like manner to those in `JuMP.jl` with the same functions. Some examples include:
- `set_lower_bound`
- `fix`
- `set_binary`
- `UpperBoundRef`
- `set_start_value`
- `delete`

The only exception is that for infinite variables and derivatives the start values are queried/modified via `start_value_function` and `set_start_value_function`.

We can also query the infinite parameters that a variable depends on via `parameter_refs`:

In [ ]:
parameter_refs(q)

#### Exercise: Create Variables
**Problem**
- Add a variable $v(\ell) \in \{0, 1\}^2$
- Create a point variable $v_1(-1)$

In [ ]:
# PUT CODE HERE


For more information (including anonymous definitions) see https://pulsipher.github.io/InfiniteOpt.jl/stable/guide/variable/.

### Differential Operators
Differential operators capture how infinite variables vary with respect to infinite parameters. `InfiniteOpt.jl` currently supports derivative operators.

These can be defined via `deriv` (alternatively, `∂` produced with `\partial`):

In [ ]:
∂(y, t) # make a first order derivative

This can even operate directly on affine/quadratic expressions:

In [ ]:
∂(2y * z - y^2 + 4y - z + 2, t)

Here the proper symbolic calculus rules are respected. We could even (unnecessarily) input an expression of infinite parameters:

In [ ]:
∂(3t^2 + z * t - 3t + 4, t)

General nonlinear expressions however are not supported, but this can be added if there is enough interest:

In [ ]:
∂(sin(y)^3, t)

Despite the error, notice that we just made a nonlinear expression without an `@NLexpression` macro, we'll talk more about that in the next section. 

To more efficiently parse large expression inputs and/or specify higher-order derivatives we can use `@deriv` (`@∂`):

In [ ]:
@∂(q, t^2, x[1])

Note the derivative operators are applied recursively. Note, that a pending development will preserve higher-order derivatives. 

We can also define derivatives via `@variable` with the `Deriv` tag if we want:

In [ ]:
@variable(model, 0 <= dy <= 1, Deriv(y, t), start = 4)

This allows us to convenient create an alias Julia variable, add bounds, and an initial guess value if wanted. We can also apply the same modification/query functions to derivatives as we can to infinite variables. For instance:

In [ ]:
fix(dy, 0, force = true)
set_lower_bound(∂(q, t), 42)

We can also create semi-infinite and/or point derivatives using the functional syntax or `@variable`:

In [ ]:
dy(0)
∂(q, x[1])(0, x)

#### Exercise: Create a Derivative
**Problem**
- Add $\frac{\partial^2}{\partial t^2}\left[4q^2(t, x) + q(t, x)\right]$

In [ ]:
# PUT CODE HERE
@deriv(4q^2 + q, t^2)

For more information see https://pulsipher.github.io/InfiniteOpt.jl/stable/guide/derivative/.

### Expressions
`InfiniteOpt.jl` inherits the same expression syntax as `JuMP.jl`. Hence, for affine/quadratic expressions we can use `@expression` as normal:

In [ ]:
@expression(model, my_expr, q^2 + y - 3)

We can also define expressions outside of macros (not recommended):

In [ ]:
my_expr = q^2 + y - 3 # defining outside macros is not as performant

One key difference with `InfiniteOpt.jl` vs. `JuMP.jl` is how we handle general nonlinear expressions. `InfiniteOpt.jl`'s nonlinear expressions permit the following:
- define nonlinear expressions outside of macros
- define expressions in the normal macros (not the `@NL` macros)
- can use linear algebra operations

For instance:

In [ ]:
@expression(model, nl_expr, sin(y)^2 / q)
nl_expr = sin(y)^2 / q

Let's see some linear algebra with nonlinear expressions:

In [ ]:
@variable(model, Q[1:2, 1:2]); @variable(model, W[1:2, 1:2]);

@expression(model, W * Q * v)

We support all the same native nonlinear functions as `JuMP.jl` with 3 caveats:
- Functions from `SpecialFunctions.jl` can only be used if `using SpecialFunctions` is included first
- The `ifelse` function must be specified `InfiniteOpt.ifelse`
- The logic operators `&` and `|` must be used instead of `&&` and `||` when defining a nonlinear expression.

For instance:

In [ ]:
using SpecialFunctions

@show y^2.3 * gamma(y)
@show InfiniteOpt.ifelse((y <= 0) | (y >= 3), y^2.3, exp(y));

We can query all the registered nonlinear functions via `all_registered_functions`:

In [ ]:
all_registered_functions(model)

Like `JuMP.jl` we can register unsupported functions. Here we use `@register`:

In [ ]:
myfunc(a) = logerfcx(a) # functions from other packages must be wrapped
@register(model, myfunc(a)) # `a` is an arbitrary symbol to indicate the argument structure
myfunc(y)

Here auto-differentiation is implied, but the gradient and the hessian functions can be given explicitly if wanted following `JuMP.jl`'s syntax.

For more information, see https://pulsipher.github.io/InfiniteOpt.jl/stable/guide/expression/.

### Measure Operators
Measure operators summarize an infinite variable/expression over a particular infinite parameter. `InfiniteOpt.jl` features a generic measure API, but more commonly users will use integrals and expectations.

Integrals and expectations can be defined with `integral` (`∫` from `\int`) and `expect` (`𝔼` from `\bbE`):

In [ ]:
𝔼(q * y + y, t) 
∫(q * t - sin(q), t)

Notice that the integral uses the full domain of the infinite parameter by default. We can truncate the domain by specifying the bounds:

In [ ]:
∫(q, t, 0, 5)

The expectation uses the appropriate pdf with random infinite parameters. With other infinite parameters (e.g., $t$) the default pdf is $\frac{1}{ub - lb}$, but we can specify a different one via the `pdf` keyword argument:

In [ ]:
𝔼(y, t, pdf = t -> exp(-t)) # here the pdf acts as a discount factor

This introduces the notion of a time-valued pdf and expectation, we'll discuss this more later. 

Similarly, we can add a weighting function (mapping the infinite parameter to a scalar) to integrals via the `weight_func` argument:

In [ ]:
∫(y, t, weight_func = t -> t^2)

For increased efficiency with handling expressions, we also provide `@integral`, `@∫`, `@expect`, and `@𝔼`.

#### Exercise: Differentiate an Integral
**Problem**
- Define $\frac{d}{dx_1}\left[\int_{t \in [0, 10]} q(t, x)^2 dt\right]$

In [ ]:
# PUT CODE HERE 


We will discuss the approximation schemes for these in the transformation section.

For more information see https://pulsipher.github.io/InfiniteOpt.jl/stable/guide/measure/.

### Objectives
We define objectives using `@objective` like normal, typically these will contain measures that scalarize the cost function to be well-posed:

In [ ]:
@objective(model, Min, ∫(sin(y), t))

Note that we can embed nonlinear expressions directly without using `@NLobjective`. 

One common mistake, is defining objectives without a fully reduced cost function (i.e., we don't measure all the infinite parameters):

In [ ]:
@objective(model, Min, ∫(q, t))

Let's try that again:

In [ ]:
@objective(model, Min, ∫(∫(q, x), t))

We can query/modify objectives just like one can in `JuMP.jl`.

For more information, see https://pulsipher.github.io/InfiniteOpt.jl/stable/guide/objective/.

### Constraints
Constraints are created in `InfiniteOpt.jl` using `@constraint`, `@NLconstraint` is not needed nor supported.

Let's try some out:

In [ ]:
@constraint(model, sin(y) + z <= 0)
@constraint(model, Q >= 0, PSDCone()) # all the special constraints from JuMP
@constraint(model, [i ∈ 1:2], v[i]^2 + 5z >= 0) 
@constraint(model, sin.(v).^3 .== 0) # nonlinear broadcasting
@constraint(model, 5z * y + q^2 <= 3)

Note that while it is possible to create nonlinear vector constraints in `InfiniteOpt.jl`, these are not currently supported by any transformation backend.

Commonly, we wish to enforce point constraints (e.g., boundary conditions), this can be accomplished using our functional point variable syntax:

In [ ]:
@constraint(model, initial, y(0) == 42)

We can also define constraints whose domain is restricted to a portion of the infinite domain by adding `DomainRestrictions`:

In [ ]:
@constraint(model, y + 2z ≤ 0, DomainRestrictions(t => [2, 5]))

For now this is limited to ranges, but development is underway to accept arbitrary logic. 

We can query/modify/delete constraints just like we can in `JuMP.jl` using the same methods. For instance:

In [ ]:
delete(model, initial)

#### Exercise: Constraints
**Problem**
- Add $\sum_{i \in \{1, 2\}} \tan(v_i(\ell)) \geq 42, \; \forall \ell \in [0, 1]$

In [ ]:
# PUT CODE HERE


For more information, see https://pulsipher.github.io/InfiniteOpt.jl/stable/guide/constraint/.

## Transformations and Solution
`InfiniteOpt` is based on a transformation paradigm. This is used to solve `InfiniteModel`s. In this section, we will highlight how this is done.

### The Optimizer Model Framework
As we highlighted above, `InfiniteModel`s contain an optimizer model backend (an augmented `JuMP.Model`) that is ultimately used to solve the infinite models.

![transform](figures/transformation.png)

Optimizer models store the transformed version of an `InfiniteModel` that can be handled by `JuMP.jl`. Moreover, they contain mapping information to support seamless interaction with `InfiniteModel`s.

Optimizer models are queried via `optimizer_model` and are populated whenever `build_optimizer_model!` is called:

In [ ]:
model = InfiniteModel(HiGHS.Optimizer)
@show optimizer_model(model)
println()

@infinite_parameter(model, t ∈ [0, 1], num_supports = 3)
@variable(model, y ≥ 0, Infinite(t))
@objective(model, Min, ∫(y, t))

build_optimizer_model!(model)

@show optimizer_model(model)

latex_formulation(optimizer_model(model))

We'll discuss the particulars of what the `TranscriptionModel` backend transformation below, but here we can see how the model is transformed and stored in the optimizer model.

Currently, we are working on generalizing this paradigm to accept arbitrary transformation backends (not just `JuMP.Model`s). See https://github.com/pulsipher/InfiniteOpt.jl/pull/248 to track the progress on this.

### Direct Transcription via `TranscriptionOpt`
The default optimizer model backend in `InfiniteOpt.jl` is `TranscriptionOpt` which can apply a suite of discretization techniques to transform `InfiniteModel`s. These all fall under the umbrella of direct transcription. This principal is illustrated below:

![transcription](figures/transcription.png)

The general methodology for applying direct transcription is the following:
1. Define supports (discretization points) for each infinite parameter
2. Add any additional supports needed for derivative/integral approximation (e.g., collocation points)
3. Generate the appropriate transcription variables
4. Expand measures via an appropriate approximation scheme (e.g., trapezoid rule)
5. Replace remaining infinite variables with transcription variables over each support combination
6. Transcribe infinite constraints over all support combinations they depend on
7. Add on auxiliary derivative approximation equations for derivative variables

For instance, consider the simple space-time model:
$$
\begin{aligned}
	&&\min_{y(t), g(t, x)} &&& \int_0^{10} y^2(t) dt \\
	&&\text{s.t.} &&& y(0) = 1 \\
	&&&&& \int_{x \in [-1, 1]^2} \frac{\partial g(t, x)}{\partial t} dx = 42, && \forall t \in [0, 10] \\
  &&&&& 3g(t, x) + 2y^2(t) \leq 2, && \forall t \in T, \ x \in [-1, 1]^2. \\
\end{aligned}
$$
Define supports for the infinite parameters $t$ and $x$:
$$
t \in \{0, 10\}, \;\;\;\;\; x \in \{[-1, -1]^T, [-1, 1]^T, [1, -1]^T, [1, 1]^T\}
$$
Now we expand the integral measures via a trapezoid rule:
$$
\begin{aligned}
	&&\min_{y(t), g(t, x)} &&& 5y^2(0) + 5y^2(10) \\
	&&\text{s.t.} &&& y(0) = 1 \\
  &&&&& g(0, x) = 0 \\
	&&&&& \frac{\partial g(t, [-1, -1])}{\partial t} + \frac{\partial g(t, [-1, 1])}{\partial t} + \frac{\partial g(t, [1, -1])}{\partial t} + \frac{\partial g(t, [1, 1])}{\partial t} = 42, && \forall t \in [0, 10] \\
  &&&&& 3g(t, x) + 2y^2(t) \leq 2, && \forall t \in T, \ x \in [-1, 1]^2. \\
\end{aligned}
$$
Now we need to transcribe the remaining infinite variables and constraints:
$$
\begin{aligned}
	&&\min_{y(t), g(t, x)} &&& 5y^2(0) + 5y^2(10) \\
	&&\text{s.t.} &&& y(0) = 1 \\
  &&&&& g(0, [-1, -1]) = 0 \\
  &&&&& g(0, [-1, 1]) = 0 \\
  &&&&& g(0, [1, -1]) = 0 \\
  &&&&& g(0, [1, 1]) = 0 \\
	&&&&& \frac{\partial g(0, [-1, -1])}{\partial t} + \frac{\partial g(0, [-1, 1])}{\partial t} + \frac{\partial g(0, [1, -1])}{\partial t} + \frac{\partial g(0, [1, 1])}{\partial t} = 42\\
  &&&&& \frac{\partial g(10, [-1, -1])}{\partial t} + \frac{\partial g(10, [-1, 1])}{\partial t} + \frac{\partial g(10, [1, -1])}{\partial t} + \frac{\partial g(10, [1, 1])}{\partial t} = 42\\
  &&&&& 3g(0, [-1, -1]) + 2y^2(0) \leq 2 \\
  &&&&& 3g(0, [-1, 1]) + 2y^2(0) \leq 2 \\
  &&&&& \vdots \\
  &&&&& 3g(10, [1, 1]) + 2y^2(10) \leq 2.
\end{aligned}
$$
Finally, we tag on the auxiliary derivative variable equations determined by backward finite difference:
$$
\begin{aligned}
&&& g(10, [-1, -1]) = g(0, [-1, -1]) + 10\frac{\partial g(10, [-1, -1])}{\partial t} \\
&&& g(10, [-1, 1]) = g(0, [-1, 1]) + 10\frac{\partial g(10, [-1, 1])}{\partial t} \\
&&& g(10, [1, -1]) = g(0, [1, -1]) + 10\frac{\partial g(10, [1, -1])}{\partial t} \\
&&& g(10, [1, 1]) = g(0, [1, 1]) + 10\frac{\partial g(10, [1, 1])}{\partial t}
\end{aligned}
$$

Let's see how `InfiniteOpt.jl` does:

In [ ]:
# Initialize model
inf_model = InfiniteModel()

# Define parameters and supports
@infinite_parameter(inf_model, t in [0, 10], supports = [0, 10])
@infinite_parameter(inf_model, x[1:2] in [-1, 1], supports = [-1, 1], independent = true)

# Define variables
@variable(inf_model, y, Infinite(t))
@variable(inf_model, g, Infinite(t, x))

# Set the objective
@objective(inf_model, Min, ∫(y^2, t))

# Define the constraints
@constraint(inf_model, y(0) == 1)
@constraint(inf_model, con, g(0, x) == 0)
@constraint(inf_model, ∫(∫(∂(g, t), x[1]), x[2]) == 42)
@constraint(inf_model, 3g + y^2 <= 2)

# Print the infinite model
latex_formulation(inf_model)

In [ ]:
build_optimizer_model!(inf_model)

opt_model = optimizer_model(inf_model)
latex_formulation(opt_model)

This exactly matches what we did by hand! The support combinations are computed automatically. We can determine their values via `supports`:

In [ ]:
@show supports(y)

supports(g)

The tuple order corresponds to the infinite parameters of each, which we can check via `parameter_refs`:

In [ ]:
@show parameter_refs(y)

@show parameter_refs(g);

A `TranscriptionModel` is not just a regular `JuMP.Model`:

In [ ]:
@show is_transcription_model(opt_model)
@show is_transcription_model(Model());

This is because a `TranscriptionModel` also contains transcription data, so we can map `inf_model` to `opt_model` behind the scenes:

In [ ]:
typeof(transcription_data(opt_model))

We can access these mappings via `transcription_variable`, `transcription_expression`, and `transcription_constraint`:

In [ ]:
@show transcription_variable(y)
@show transcription_expression(y^2 + 2)
transcription_constraint(con)

#### Integral Approximations
A wide variety of integral approximations (14 in total) are currently supported. These include:
- Trapezoid rule (default)
- Gaussian quadrature
- Monte Carlo sampling

The full list is provided at https://pulsipher.github.io/InfiniteOpt.jl/stable/guide/measure/#Evaluation-Methods.

Let's try specifying an integral that uses an appropriate Gauss quadrature scheme:

In [ ]:
model = InfiniteModel()
@infinite_parameter(model, t ∈ [0, 1])
@variable(model, y, Infinite(t))

my_int = ∫(y, t, eval_method = Quadrature(), num_nodes = 4)

That's it, `InfiniteOpt.jl` has a sophisticated support management system to properly keep track of where all supports come from and where they will be needed for transcription. 

We can interrogate what the approximation will look like using `expand`:

In [ ]:
expand(my_int)

Note that `expand` should be avoided with models you intent to solve, since point/semi-infinite variables are added to the model prematurely to facilitate the preview. This is only intended as a method for curious users.

#### Derivative Approximations
For consistency, derivative approximation methods are associated with the infinite parameters the derivatives depend on. We can specify them when adding infinite parameters or later on via `set_derivative_method`. Currently, 4 such methods are supported:
- Forward finite difference
- Central finite difference
- Backward finite difference (default)
- Orthogonal collocation over finite elements with Lobatto quadrature

To set orthogonal collocation with 3 nodes per finite element, we would write:

In [ ]:
model = InfiniteModel()
@infinite_parameter(model, t ∈ [0, 1], num_supports = 3, derivative_method = OrthogonalCollocation(3))
@variable(model, y, Infinite(t))

@constraint(model, ∂(y, t) >= 0)
@constraint(model, y(0) == 0);

This will use 3 supports to construct the finite element boundaries (giving 2 finite elements). Later on collocation points will be added (1 extra per element). We can preview the derivative approximations via `evaluate_all_derivatives!` and `derivative_constraints`:

In [ ]:
evaluate_all_derivatives!(model)
derivative_constraints(∂(y, t))

Again, these modify `model` in place and should not be used with models we actually want to solve.

For more information, see https://pulsipher.github.io/InfiniteOpt.jl/stable/guide/derivative/#Derivative-Evaluation.

### Transcription Solutions
Querying the solution of an `InfiniteModel` is done in similar manner to `JuMP` models.

Let's begin by creating and solving a model:

In [ ]:
model = InfiniteModel(HiGHS.Optimizer)
set_silent(model)
@infinite_parameter(model, t in [0, 10], num_supports = 10)
@variable(model, y >= 0, Infinite(t))
@variable(model, z >= 0)
@objective(model, Min, 2z)
@constraint(model, c1, z >= y)
@constraint(model, c2, y(0) == 42)
optimize!(model)

We can do all the model based queries in exactly the same way to `JuMP`:

In [ ]:
@show termination_status(model)
@show primal_status(model)
@show dual_status(model)
@show has_values(model)
@show objective_value(model)
@show solve_time(model);

We can query the values of the variables and constraints using `value`

In [ ]:
@show value(z)
@show value(y)
@show value(c1)
@show value(c2);

These correspond to the values of the transcription variables/constraints.

We can also query the constraints as normal with methods like `dual` and `shadow_price`:

In [ ]:
@show shadow_price(c1)
@show shadow_price(c2);

For linear models, we can even get the sensitivity report:

In [ ]:
report = lp_sensitivity_report(model)
@show report[c2]
report[c1]

We can also interrogate the transcription model directly:

In [ ]:
solution_summary(transcription_model(model))

### Other Solution Approaches
As we will discuss in the last module today, `InfiniteOpt.jl` is modular and new transformation approaches can readily be added.

Currently, `TranscriptionOpt` is the only backend available. 